<a href="https://colab.research.google.com/github/slowvak/MC-ImageWrangling/blob/master/ImageWrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Cell #1
# The first cell (run Cell #1) loads in a few data sets of interest, including: T1, T2, GAD on several subjects with brain tumors. 
# these are DICOM images, so we have to first convert them to nifti
# and the nifti converter leaves the output in the source folder, so we have
# to move them and also rename them. Will use python for that

!rm -rf MC-ImageWrangling
!git clone https://github.com/slowvak/MC-ImageWrangling.git
  
!unzip -q MC-ImageWrangling/Bet-n-dcm2nii.zip
!rm -rf images

!mkdir images
!cd images; unzip -q "../MC-ImageWrangling/S1-4.zip" 
!cd images; unzip -q "../MC-ImageWrangling/S5-8.zip" 
!cd images; unzip -q "../MC-ImageWrangling/S9-12.zip" 
!cd images; unzip -q "../MC-ImageWrangling/S13-16.zip" 

!rm -rf nii
!rm -rf tmp

!mkdir tmp
!mkdir nii

NIFTI_PATH = './nii'
# now some python code to loop over all the folders and convert from DICOM to nifti
# and then since the command doesn't allow speciyfing the name, will also move
#them from temp directory to an organized directory.
# this is why seemingly simple tasks always take longer than planned...

import os, fnmatch

for subj in fnmatch.filter(os.listdir('./images'), 'S*'):
    subj_path = os.path.join('./images', subj)
    series = os.listdir(subj_path)
    series = ['T1', 'T2', 'GAD']
    for ser in series:
        dcm_path = os.path.join (subj_path, ser)
        cmd = "./dcm2nii -o tmp %s" % (dcm_path)
        os.system(cmd)
#        print (cmd)
        # now have to find tehe .nii.gz and move it out
        f = fnmatch.filter(os.listdir('./tmp'), '*.nii.gz')
        out_dir = os.path.join (NIFTI_PATH, subj)
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
        out_path = os.path.join (out_dir, ser)
        cmd = "mv ./tmp/%s %s.nii.gz" % (f[0], out_path)
        os.system(cmd)
#        print (cmd)


Cloning into 'MC-ImageWrangling'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 41 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (41/41), done.
Checking out files: 100% (24/24), done.
replace dcm2nii? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
total 5888
-rw-r--r-- 1 root root 2023457 Jul  4 17:47 GAD.nii.gz
-rw-r--r-- 1 root root 1951148 Jul  4 17:47 T1.nii.gz
-rw-r--r-- 1 root root 2044295 Jul  4 17:47 T2.nii.gz


In [0]:
# CELL #2
# Now do some basic processing to prepare the images
# Note this will take about 30 minutes to complete.
# If you want to skip it, the output is already computed and saved for next cell
#!ls ./nii
#!ls ./nii/S1

# FSL (flirt-the image registration application) complains if this environment variable is not set
os.environ['FSLOUTPUTTYPE'] = 'NIFTI_GZ'

#for subj in ['S1','S2']:
for subj in os.listdir(NIFTI_PATH):
    subj_path = os.path.join(NIFTI_PATH, subj)
    print ("Working on " + subj_path)
    series = fnmatch.filter(os.listdir(subj_path), '*.nii.gz')
    for ser in ['T1', 'T2', 'GAD']:
        nii_file = os.path.join (subj_path, ser)
# First, perform N4 bias correction. Not required, but may improve results. Also must track new names
        new_file = os.path.join (subj_path, 'N4-' + ser)
        cmd = "./N4BiasFieldCorrection -i %s.nii.gz -o %s.nii.gz" % (nii_file, new_file)
        os.system(cmd)
#        print (cmd)
# Next, Register the images to the post-Gad images so skip the GAD
    GAD_file = os.path.join (subj_path, 'N4-GAD.nii.gz')
    T1_file =  os.path.join (subj_path, 'N4-T1.nii.gz')
    new_file = os.path.join (subj_path, 'Reg-T1.nii.gz')
    cmd = "./flirt -in %s -ref %s  -out %s" % (T1_file, GAD_file, new_file)
    os.system(cmd)
#    print (cmd)
    T2_file =  os.path.join (subj_path, 'N4-T2.nii.gz')
    new_file = os.path.join (subj_path, 'Reg-T2.nii.gz')
    cmd = "./flirt -in %s -ref %s  -out %s" % (T2_file, GAD_file, new_file)
    os.system(cmd)
#    print (cmd)
# can also try to mask out non-brain tissue, but that doesn't work as well on thick slice MRI

# finally, copy over original so you can skip these steps if you like
    cmd = "cp %s.nii.gz %s.nii.gz" % (os.path.join (subj_path, 'Reg-T1'), os.path.join (subj_path, 'T1'))
    os.system(cmd)
    cmd = "cp %s.nii.gz %s.nii.gz" % (os.path.join (subj_path, 'Reg-T2'), os.path.join (subj_path, 'T2'))
    os.system(cmd)
    cmd = "cp %s.nii.gz %s.nii.gz" % (os.path.join (subj_path, 'N4-GAD'), os.path.join (subj_path, 'GAD'))
    os.system(cmd) 

!ls -l ./nii/S1        


Working on ./nii/S12
Working on ./nii/S9
Working on ./nii/S2
Working on ./nii/S6
Working on ./nii/S13
Working on ./nii/S3
Working on ./nii/S7
Working on ./nii/S11
Working on ./nii/S10
Working on ./nii/S4
Working on ./nii/S5
Working on ./nii/S16
Working on ./nii/S14
Working on ./nii/S1
Working on ./nii/S15
Working on ./nii/S8
total 43516
-rw-r--r-- 1 root root 5549347 Jul  4 18:09 GAD.nii.gz
-rw-r--r-- 1 root root 5549347 Jul  4 18:08 N4-GAD.nii.gz
-rw-r--r-- 1 root root 5564294 Jul  4 18:07 N4-T1.nii.gz
-rw-r--r-- 1 root root 5700586 Jul  4 18:07 N4-T2.nii.gz
-rw-r--r-- 1 root root 5472642 Jul  4 18:08 Reg-T1.nii.gz
-rw-r--r-- 1 root root 5617556 Jul  4 18:09 Reg-T2.nii.gz
-rw-r--r-- 1 root root 5472642 Jul  4 18:09 T1.nii.gz
-rw-r--r-- 1 root root 5617556 Jul  4 18:09 T2.nii.gz
total 40552
-rw-r--r-- 1 root root 5183805 Jul  4 17:53 GAD.nii.gz
-rw-r--r-- 1 root root 5183805 Jul  4 17:52 N4-GAD.nii.gz
-rw-r--r-- 1 root root 5182360 Jul  4 17:51 N4-T1.nii.gz
-rw-r--r-- 1 root root 53467

In [0]:
# Cell #3

# This unzips known good files processed as above.
# skip this cell if you want to use your version

for subj in ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16']:
    cmd = 'cd ./nii; unzip -q ../MC-ImageWrangling/%s.zip .' % (subj)
    os.system(cmd)
# must also delete the DICOM and intermediate files to have enough space to make the new files
!rm -rf nii/Reg*
!rm -rf nii/N4-*
!rm -rf ./images
!rm T1.zip
!rm T2.zip
!rm GAD.zip


# At this point, we have nifti files (T1.nii.gz, T2.nii.gz, GAD.nii.gz) for each subject


In [0]:
# Cell #4

# There is also a text file called 'TumorSlices.csv' that I created which has subject ID, 
# first slice with contrast enhancement and last slice with contrast enhancement.
# note that subjects 2 and 14 don't have much enhancement
# we will read this into a Pandas dataframe
# Pandas is very popular for data analysis, and has a built-in function to read CSV (and excel) files
import pandas as pd
df = pd.read_csv ('./MC-ImageWrangling/TumorSlices.csv')
print (df)

   Subject  StartSlice  EndSlice
0       S1          11        18
1       S2           7         8
2       S3          12        15
3       S4          12        18
4       S5           8        13
5       S6          10        17
6       S7          14        21
7       S8          11        18
8       S9           7        12
9      S10           9        15
10     S11          10        19
11     S12           7        11
12     S13          10        17
13     S14           7        11
14     S15          11        15
15     S16          10        16


In [0]:
# Cell #5
# create new versions of the T1, GAD, and T2 images that range from 0 to 255, and where the 0 intensity value maps to the 5th percentile value 
# and 255 maps to the 95th percentile value.
import imageio

MIN_MR = 20

def rescale_5_95_percentile(image):
    dims = np.shape(image)
    image_1d = image.reshape(1, image.size)
    above = np.where(image_1d>MIN_MR)
    if above.size < image_1d.size/2:
        above = image_1d # avoid bad MIN_MR thresholds
    sorted_im = np.sort(above, axis = None)
    
    start = int(sorted_im.size / 20)  # 20 = 5%
    end = int(sorted_im.size * 19 / 20) # 95%
    start_val = sorted_im[start]
    end_val = sorted_im[end]
    
    image = np.maximum(image, start_val) # map values below the end_val to end_val
    image = np.minimum(image, end_val)   # map values above the end_val to end_val
    image = image - start_val # subtract the starting intensity

 #   print ("rescaling range " + str(newmin) + " to " + str(histo[1][index]))
    image = (np.maximum(image, 0) / image.max()) * 255.0
    return image

def nifti_to_img(red_file, green_file, blue_file, not_dir, yes_dir, startSlice, endSlice, subject):
### red_ green_ and blue_files are the nifit files that will be combined to go into the PNG
### not_dir is the direcotry where PNGs are stored if they are not in the (inclusive)
### range set by startSlice/endSlice
### yes_dir is where they are stored when they ARE in the range
### subject is the ID so that we can track where images came from

    try:
        nifti = nib.load(red_file)
        nif_header = nifti.header
        red_image = nifti.get_fdata()
    except:
        return 0

    try:
        if os.path.isfile(green_file):
            nifti = nib.load(green_file)
            nif_header = nifti.header
            green_image = nifti.get_fdata()
        else:
            pass
    except:  # if can't load, then use red image
        green_image = red_image
    
    try:
        if os.path.isfile(blue_file):
            nifti = nib.load(blue_file)

            nif_header = nifti.header
            blue_image = nifti.get_fdata()
        else:
            pass
    except:  # if can't load, then use red image
        blue_image = red_image
    
    red_image = rescale_5_95_percentile(red_image)
    green_image = rescale_5_95_percentile(green_image)
    blue_image = rescale_5_95_percentile(blue_image)
    dims = np.shape(red_image)
    zd = dims[2]
    red_image = red_image.astype(np.uint8)     
    green_image = green_image.astype(np.uint8)     
    blue_image = blue_image.astype(np.uint8)     
    for z in range(0, zd):
        outname = subject + "-{0:04d}".format(img) + '.png'
        if z >= startSlice and z <= endSlice:  # this is enhancing
            out_dir = yes_dir
        else:
            out_dir = not_dir
        outname = os.path.join(out_dir, outname)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        combined = np.dstack((red_image[:,:,z],green_image[:,:,z],
                              blue_image[:,:,z]))
## . images come in rotated
#        combined = np.rot90(combined, k=3)
        print ('Name: ')
        imageio.imwrite(outname, combined)
    return zd



In [0]:
# Cell #6
# take the GAD nii, scale it to 0-255 and store that into the red channel
# same for T1 (green) and T2 (blue)
# PNG files are stored in enh / nonenh folders based on the csv file

# store the png files into enhancing or nonenhancing folders after making sure they are cleared

!rm -rf enhancing
!rm -rf nonenhancing

!mkdir enhancing
!mkdir nonenhancing

for subj in os.listdir(NIFTI_PATH):
    subj_path = os.path.join(NIFTI_PATH, subj)
    red_f = os.path.join(subj_path, 'GAD.nii.gz')
    green_f = os.path.join(subj_path, 'T1.nii.gz')
    blue_f = os.path.join(subj_path, 'T2.nii.gz')
    startSlice = df.loc[df.Subject == subj, 'StartSlice'].values[0]
    endSlice = df.loc[df.Subject == subj, 'EndSlice'].values[0]
    print ("Making PNG file for " + subj + " start: " + str(startSlice) + " end: " + str(endSlice))
    count = nifti_to_img(red_f, green_f, blue_f, './nonehancing', './enhancing', startSlice, endSlice, subj)
    

Making PNG file for S12 start: 7 end: 11
Making PNG file for S9 start: 7 end: 12
Making PNG file for S2 start: 7 end: 8
Making PNG file for S6 start: 10 end: 17
Making PNG file for S13 start: 10 end: 17
Making PNG file for S3 start: 12 end: 15
Making PNG file for S7 start: 14 end: 21
Making PNG file for S11 start: 10 end: 19
Making PNG file for S10 start: 9 end: 15
Making PNG file for S4 start: 12 end: 18
Making PNG file for S5 start: 8 end: 13
Making PNG file for S16 start: 10 end: 16
Making PNG file for S14 start: 7 end: 11
Making PNG file for S1 start: 11 end: 18
Making PNG file for S15 start: 11 end: 15
Making PNG file for S8 start: 11 end: 18


In [0]:
# Cell #7
# display some images to see the result